In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException, TimeoutException
import logging
from bs4 import BeautifulSoup
import time
import pandas as pd


class WebScraper_NewsGoogle:
    def __init__(self, keyword, start_date, end_date):
        self.keyword = keyword
        self.start_date = start_date
        self.end_date = end_date
        self.driver = None
        self.news_data = []
        
        self.setup_driver()
        self.search_for_keyword()
        self.click_on_news()
        self.click_on_tools()
        self.set_custom_interval()

    def setup_driver(self):
        
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--headless")  # Add this line to run in headless mode
        self.driver = webdriver.Chrome(options=chrome_options)       
        self.driver.get("http://www.google.com/")
        
        try:
            WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.NAME, 'q')))
        except WebDriverException:
            print("Tweets did not appear! Proceeding after timeout")

    def search_for_keyword(self):
        
        search_box = self.driver.find_element(By.NAME, 'q')
        search_box.send_keys(self.keyword)
        search_box.submit()

    def click_on_news(self):
        element = self.driver.find_element(By.LINK_TEXT, "Notícias")
        element.click()
        
    def click_on_tools(self):
        element_tools = self.driver.find_element(By.ID, "hdtb-tls")
        element_tools.click()
        
    def set_custom_interval(self):
        
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tn_1 > span:nth-child(3) > g-popup")))
        element_tools = self.driver.find_element(By.CSS_SELECTOR, "#tn_1 > span:nth-child(3) > g-popup")
        time.sleep(5) 
        element_tools.click()


        # Espera até que o elemento do menu personalizado seja clicável
        custom_menu_item = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#lb > div > g-menu > g-menu-item:nth-child(8) > div > div > span"))
        )
        custom_menu_item.click()

        search_de = self.driver.find_element(By.CLASS_NAME, 'OouJcb')
        search_de.send_keys(self.start_date)

        search_a = self.driver.find_element(By.CLASS_NAME, 'rzG2be')
        search_a.send_keys(self.end_date)

        element_tools = self.driver.find_element(By.CSS_SELECTOR, "#T3kYXe > g-button")
        element_tools.click()
        

    def scrape_data(self):
        News_data = []
        try:
            while True:
                try:
                    # Tente encontrar os resultados de notícias usando o seletor CSS
                    try:
                        news_results = self.driver.find_elements(By.CSS_SELECTOR, 'div#rso > div > div > div > div')
                        for news_div in news_results:   
                            try:
                                news_item = {}

                                try:
                                    news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                                    news_item["Link"] = news_link
                                except NoSuchElementException as e:
                                    print("Link not found:", e)
                                    logging.error("Link not found: " + str(e))
                                    news_item["Link"] = None
                                    continue  # Continue para o próximo item de notícia

                                # Extrair o domínio
                                domain_elements = news_div.find_elements(By.CSS_SELECTOR, 'a > div > div > div')

                                if len(domain_elements) >= 2:
                                    domain = domain_elements[1].text
                                    news_item["Domain"] = domain
                                else:
                                    print("Domain not found")
                                    news_item["Domain"] = None

                                # Extrair o título
                                if len(domain_elements) >= 3:
                                    title = domain_elements[2].text
                                    news_item["Title"] = title
                                else:
                                    print("Title not found")
                                    news_item["Title"] = None

                                # Extrair a descrição
                                if len(domain_elements) >= 4:
                                    description = domain_elements[3].text
                                    news_item["Description"] = description
                                else:
                                    print("Description not found")
                                    news_item["Description"] = None

                                # Extrair a data
                                if len(domain_elements) >= 5:
                                    date = domain_elements[4].text
                                    news_item["Date"] = date
                                else:
                                    try:
                                        Date = news_div.find_element(By.CSS_SELECTOR, '#rso > div > div > div.SoaBEf.R24aHf > div > div > a > div > div.iRPxbe > div.OSrXXb.rbYSKb.LfVVr')
                                        news_item["Date"] = Date
                                    except NoSuchElementException as e:
                                        #print("Date not found")
                                        news_item["Date"] = None
                                        continue  # Continue para o próximo item de notícia
                                   


                                #print("Link:", news_item["Link"])
                                #print("Domain:", news_item["Domain"])
                                #print("Title:", news_item["Title"])
                                #print("Description:", news_item["Description"])
                                #print("Date:", news_item["Date"])
                                #print("-" * 50 + "\n\n" + "-" * 50)

                                try:
                                    News_data.append(news_item)

                                except Exception as e:
                                    print("Error appending news_item to News_data:", e)
                                    logging.error("Error appending news_item to News_data: " + str(e))


                            except NoSuchElementException as e:
                                print("Incomplete news item:", e)
                                logging.error("Incomplete news item: " + str(e))

                                try:                                
                                    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, "pnnext")))
                                    elemento_proxima = self.driver.find_element(By.ID, "pnnext")
                                    #time.sleep(2)
                                    elemento_proxima.click()

                                except NoSuchElementException:
                                    # Tratamento de erro: O botão "Próxima" não foi encontrado, indicando o fim das páginas
                                    print("Não há mais páginas disponíveis.")


                    except NoSuchElementException as e:
                        print("News results not found:", e)
                        logging.error("News results not found: " + str(e))
                        # Sair do loop se os resultados de notícias não forem encontrados

                except NoSuchElementException:
                    # Tratamento de erro: O elemento "Próxima" não foi encontrado, ou seja, não há mais páginas
                    print("Não há mais páginas disponíveis.")

                # Encontre o elemento "Próxima" (ou equivalente) pelo ID ou outro seletor
                try:
                    elemento_proxima = self.driver.find_element(By.ID, "pnnext")
                    elemento_proxima.click()

                except NoSuchElementException:
                    # Tratamento de erro: O botão "Próxima" não foi encontrado, indicando o fim das páginas
                    print("Não há mais páginas disponíveis.")
                    break

        except TimeoutException as e:
            print(f"Timeout Exception: {e}")
            logging.error("Timeout Exception: " + str(e))

        except Exception as e:
            print(f"An error occurred: {e}")
            logging.error("An error occurred: " + str(e))

        #finally:
            # Feche o navegador após a conclusão
            #driver.quit()

        return News_data

    def close_driver(self):
        if self.driver:
            self.driver.quit()
            print("Realizada")

if __name__ == "__main__":
    print("Pronto Para começar!")
    #keyword = 'AAPL'
    #start_date = '6/1/2015'
    #end_date = '7/1/2015'
    
    #Google_news = WebScraper_NewsGoogle(keyword, start_date, end_date)
    #News = Google_news.scrape_data()


Pronto Para começar!


In [ ]:
#Código para raspagem por período

In [ ]:
data_list = []

In [ ]:
keyword = 'AAPL'
start_date = '6/1/2015'
end_date = '7/1/2015'
    
Google_news = WebScraper_NewsGoogle(keyword, start_date, end_date)
News = Google_news.scrape_data()
data_list.extend(News)
Google_news.close_driver()

In [ ]:
df = pd.DataFrame(data_list)

In [ ]:
df.to_excel('Bitcoin_Google_Browser_News20196.xlsx', index=False)  # O parâmetro index=False evita que o índice seja incluído no arquivo Excel 


In [ ]:
#mensal

In [ ]:
from datetime import datetime, timedelta

start_date = datetime(2015, 1, 1)
# Defina a data final como a data de hoje
end_date = datetime(2016, 1, 1)

# Defina a palavra-chave
keyword = 'Bitcoin News'

# Lista para armazenar os dados mensalmente
data_list = []

# Loop mensal
while start_date <= end_date:
    # Calcule a data de início do mês seguinte
    next_month = start_date.replace(day=1) + timedelta(days=32)
    next_month = next_month.replace(day=1)
    
    # Converta as datas para strings no formato desejado
    start_date_str = start_date.strftime('%m/%d/%Y')
    next_month_str = next_month.strftime('%m/%d/%Y')
    
    # Crie uma instância da classe e extraia os dados
    Google_news = WebScraper_NewsGoogle(keyword, start_date_str, next_month_str)
    news_data = Google_news.scrape_data()
    Google_news.close_driver()
    
    # Adicione os dados à lista
    data_list.extend(news_data)
    
    # Atualize a data de início para o próximo mês
    start_date = next_month


In [5]:
df = pd.DataFrame(data_list)

In [52]:
df.to_excel('Bitcoin_Google_Browser_News20196.xlsx', index=False)  # O parâmetro index=False evita que o índice seja incluído no arquivo Excel 


In [ ]:
#semanal

In [ ]:
from datetime import datetime, timedelta

# Defina a data de início em janeiro de 2015
start_date = datetime(2015, 2, 1)

# Defina a data final como a data de hoje
end_date = datetime(2015, 3, 1)

# Defina a palavra-chave
keyword = 'Bitcoin News'

# Lista para armazenar os dados semanalmente
data_list = []

# Loop semanal
while start_date <= end_date:
    # Calcule a data da próxima semana
    next_week = start_date + timedelta(days=7)

    # Converta as datas para strings no formato desejado
    start_date_str = start_date.strftime('%m/%d/%Y')
    next_week_str = next_week.strftime('%m/%d/%Y')

    # Crie uma instância da classe e extraia os dados
    Google_news = WebScraper_NewsGoogle(keyword, start_date_str, next_week_str)
    news_data = Google_news.scrape_data()
    Google_news.close_driver()

    # Adicione os dados à lista
    data_list.extend(news_data)

    # Atualize a data de início para a próxima semana
    start_date = next_week
    
    time.sleep(5)

In [4]:
df = pd.DataFrame(data_list)

In [5]:
df

,Link,Domain,Title,Description,Date
0,https://www.zdnet.com/article/bitstamp-bitcoin...,ZDNET,"Bitstamp exchange hacked, $5M worth of bitcoin...",... bitcoin.png. Bitstamp prices plummeted aft...,5 de jan. de 2015
1,https://www.coindesk.com/markets/2015/01/05/bi...,CoinDesk,Bitstamp Claims $5 Million Lost in Hot Wallet ...,... bitcoin deposit addresses. As an additiona...,5 de jan. de 2015
2,https://www.ft.com/content/1c6217d9-8070-30af-...,Financial Times,Who needs caveat emptor when you've got Bitcoin?,News updates from October 10: Birkenstock pric...,5 de jan. de 2015
3,https://www.businessinsider.com/bitcoin-jesus-...,Business Insider,'Bitcoin Jesus' Visa Application Denied,... News · Politics · Military & Defense · Spo...,7 de jan. de 2015
4,https://www.coindesk.com/markets/2015/01/08/ap...,CoinDesk,Apple Approves iOS Game That Tips Players in B...,The leader in news and information on cryptocu...,8 de jan. de 2015
...,...,...,...,...,...
988,https://www.cnnturk.com/turkiye/boydak-holding...,CNN Türk,"Boydak Holding işçileri iş bıraktı, eylemde",Abone ol Google News'de Paylaş FlipBoard'da Pa...,5 de fev. de 2015
989,https://www.cnnturk.com/dunya/ermeni-cemaatind...,CNN Türk,Ermeni cemaatinden ''Türk dizisi yayından kald...,Şili ve Kolombiya'da Türk dizilerinin reyting ...,29 de jan. de 2015
990,https://onedio.com/haber/sosyal-medya-gundemi-...,Onedio,Sosyal Medya Gündemi: Bank Asya'nın Açılışı,"Bitcoin · Finans. Dizi & Film, TV. Dizi & Film...",4 de fev. de 2015
991,https://onedio.com/haber/saskinliktan-henuz-at...,Onedio,Şaşkınlıktan Henüz Atom Çekirdeği Boyutundaki ...,"Bitcoin · Finans. Dizi & Film, TV. Dizi & Film...",5 de fev. de 2015


In [6]:
df.to_excel('Bitcoin_Google_Browser_News20151.xlsx', index=False)  # O parâmetro index=False evita que o índice seja incluído no arquivo Excel 
